<a href="https://colab.research.google.com/github/bobbypaton/bobbypaton/blob/main/solubility_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Computationally predicting molecular solubility is useful for drug-discovery.

In this tutorial, we will fit a simple statistical model that predicts the solubility of some organic molecules. The process of fitting this model involves four steps:

1. Loading a chemical dataset, consisting of a series of compounds along with aqueous solubility measurements.
2. Visualizing the molecules
3. Exploring how single variables influence solubility.
4. Fitting a simple multivariate model.
5. Visualizing the results

In [ ]:
import sys
sys.version

In [ ]:
!time pip install rdkit-pypi

In [ ]:
# import python modules
import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import sklearn, sys
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

import rdkit
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_useSVG=True

We need to load a dataset of estimated aqueous solubility measurements [1] into our notebook. The data is in CSV format and contains SMILES strings, measured aqueous solubilities, and other descriptors.

In [ ]:
solubility_data = pd.read_csv("https://raw.githubusercontent.com/deepchem/deepchem/master/datasets/delaney-processed.csv")
solubility_data = solubility_data[['Compound ID', 'smiles', 'measured log solubility in mols per litre', 'Molecular Weight', 'Minimum Degree', 'Number of Rings', 'Number of Rotatable Bonds', 'Polar Surface Area']]
solubility_data.head()

Let's take a look at some of the molecules in our dataset!



In [ ]:
molecules = [rdkit.Chem.MolFromSmiles(smi) for smi in solubility_data['smiles']]
Draw.MolsToGridImage(molecules, molsPerRow=5, legends=[str(sol) for sol in solubility_data['measured log solubility in mols per litre']])

Which are the most and least soluble compounds in our dataset?

In [ ]:
solubility_data['measured log solubility in mols per litre'].max()

In [ ]:
solubility_data['measured log solubility in mols per litre'].idxmax()

In [ ]:
solubility_data.loc[603]

In [ ]:
smiles = [solubility_data.loc[605]['smiles'], solubility_data.loc[603]['smiles']]
molecules = [rdkit.Chem.MolFromSmiles(smi) for smi in smiles]
Draw.MolsToGridImage(molecules)

Does solubility correlate with molecular structure?

In [ ]:
X = solubility_data['Molecular Weight']
Y = solubility_data['measured log solubility in mols per litre']

plt.scatter(X, Y)
plt.xlabel("Molecular Weight")
plt.ylabel("Log Solubility")
plt.show()

In [ ]:
X = solubility_data['Number of Rings']
Y = solubility_data['measured log solubility in mols per litre']

plt.scatter(X, Y)
plt.xlabel("Number of Rings")
plt.ylabel("Log Solubility")
plt.show()

Can we combine the separate variables (molecular weight, number of rings, number of rotatable bonds, polar surface area) in a way to predict the solubility? This is known as a "multivariate model".

In [ ]:
X = -0.013 * solubility_data['Molecular Weight'] -0.46 * solubility_data['Number of Rings'] - 0.15 * solubility_data['Number of Rotatable Bonds'] + 0.03 * solubility_data['Polar Surface Area'] 
Y = solubility_data['measured log solubility in mols per litre']

plt.scatter(X, Y)
plt.xlabel("Predicted Log Solubility")
plt.ylabel("Log Solubility")
plt.show()
r2_score(Y, X)